In [30]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import r2_score,accuracy_score, confusion_matrix, roc_auc_score
from sklearn.model_selection import cross_val_score, cross_validate, StratifiedKFold, GridSearchCV

In [31]:
ls

 Volume in drive C has no label.
 Volume Serial Number is 0C74-9C02

 Directory of C:\Users\Jory Anderson\Desktop\Programming\Repos\Loan-Default\notebooks

2020-07-07  08:41 PM    <DIR>          .
2020-07-07  08:41 PM    <DIR>          ..
2020-07-07  07:59 PM    <DIR>          .ipynb_checkpoints
2019-12-11  03:58 AM        26,567,651 application_test.csv
2019-12-11  03:58 AM       166,133,370 application_train.csv
2020-07-07  07:51 PM            75,769 Data_Exploration.ipynb
2020-07-07  08:36 PM            48,514 Data_Exploration_FullSet.ipynb
2020-07-07  08:41 PM           125,783 Project Exploration.ipynb
               5 File(s)    192,951,087 bytes
               3 Dir(s)  33,824,534,528 bytes free


In [32]:
dataset = pd.read_csv("application_train.csv")

In [33]:
dataset.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [34]:
dataset.shape

(307511, 122)

In [35]:
null_rows = dataset[dataset.isnull().any(axis=1)]

In [36]:
null_rows

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,456251,0,Cash loans,M,N,N,0,157500.0,254700.0,27558.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
307507,456252,0,Cash loans,F,N,Y,0,72000.0,269550.0,12001.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
307508,456253,0,Cash loans,F,N,Y,0,153000.0,677664.0,29979.0,...,0,0,0,0,1.0,0.0,0.0,1.0,0.0,1.0
307509,456254,1,Cash loans,F,N,Y,0,171000.0,370107.0,20205.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
null_rows.shape

(298909, 122)

What percent of the rows in the data have a null value in at least one column?

In [41]:
null_rows.shape[0]/dataset.shape[0]

0.9720270169197199

97% of the rows have at least one column with a null value. 
How will we handle null values?

In [42]:
dataset.columns

Index(['SK_ID_CURR', 'TARGET', 'NAME_CONTRACT_TYPE', 'CODE_GENDER',
       'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL',
       'AMT_CREDIT', 'AMT_ANNUITY',
       ...
       'FLAG_DOCUMENT_18', 'FLAG_DOCUMENT_19', 'FLAG_DOCUMENT_20',
       'FLAG_DOCUMENT_21', 'AMT_REQ_CREDIT_BUREAU_HOUR',
       'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_WEEK',
       'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_QRT',
       'AMT_REQ_CREDIT_BUREAU_YEAR'],
      dtype='object', length=122)

In [43]:
df = dataset[['TARGET', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN', 
            'AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY', 'EXT_SOURCE_1', 'EXT_SOURCE_2',
              'EXT_SOURCE_3', 'DAYS_BIRTH', 'DAYS_EMPLOYED']]

In [44]:
df

,TARGET,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,DAYS_BIRTH,DAYS_EMPLOYED
0,1,N,Y,0,202500.0,406597.5,24700.5,0.083037,0.262949,0.139376,-9461,-637
1,0,N,N,0,270000.0,1293502.5,35698.5,0.311267,0.622246,NaN,-16765,-1188
2,0,Y,Y,0,67500.0,135000.0,6750.0,NaN,0.555912,0.729567,-19046,-225
3,0,N,Y,0,135000.0,312682.5,29686.5,NaN,0.650442,NaN,-19005,-3039
4,0,N,Y,0,121500.0,513000.0,21865.5,NaN,0.322738,NaN,-19932,-3038
...,...,...,...,...,...,...,...,...,...,...,...,...
307506,0,N,N,0,157500.0,254700.0,27558.0,0.145570,0.681632,NaN,-9327,-236
307507,0,N,Y,0,72000.0,269550.0,12001.5,NaN,0.115992,NaN,-20775,365243
307508,0,N,Y,0,153000.0,677664.0,29979.0,0.744026,0.535722,0.218859,-14966,-7921
307509,1,N,Y,0,171000.0,370107.0,20205.0,NaN,0.514163,0.661024,-11961,-4786


In [45]:
df.shape

(307511, 12)

In [46]:
null_df = df[df.isnull().any(axis=1)]

In [47]:
null_df.shape

(197929, 12)

In [48]:
df = df.dropna()

In [49]:
df.shape

(109582, 12)

In [50]:
df.head()

,TARGET,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,DAYS_BIRTH,DAYS_EMPLOYED
0,1,N,Y,0,202500.000,406597.5,24700.5,0.083037,0.262949,0.139376,-9461,-637
6,0,Y,Y,1,171000.000,1560726.0,41301.0,0.774761,0.724000,0.492060,-13778,-3130
8,0,N,Y,0,112500.000,1019610.0,33826.5,0.587334,0.205747,0.751724,-20099,365243
10,0,N,Y,1,112500.000,652500.0,21177.0,0.319760,0.651862,0.363945,-10197,-679
11,0,N,Y,0,38419.155,148365.0,10678.5,0.722044,0.555183,0.652897,-20417,365243


In [51]:
# Create a label encoder object
le = LabelEncoder()
le_count = 0

# Get  columns whose data type is object i.e. string
filteredColumns = df.dtypes[df.dtypes == np.object]
# list of columns whose data type is object i.e. string
listOfColumnNames = list(filteredColumns.index)
print(listOfColumnNames)

# Label Encoding if there are two categorical labels on a feature)
for col in listOfColumnNames:
    if df.dtypes[col] == np.object:
        # If 2 or fewer unique categories
        if len(list(df[col].unique())) <= 2:
            # Train on the training data
            le.fit(df[col])
            le_count += 1
            
print('%d columns were label encoded.' % le_count)

# One-Hot Encoding (> 2 categorical labels for a feature)
numerical_df = pd.get_dummies(df)

['FLAG_OWN_CAR', 'FLAG_OWN_REALTY']
2 columns were label encoded.


In [52]:
numerical_df.head()

,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_OWN_CAR_N,FLAG_OWN_CAR_Y,FLAG_OWN_REALTY_N,FLAG_OWN_REALTY_Y
0,1,0,202500.000,406597.5,24700.5,0.083037,0.262949,0.139376,-9461,-637,1,0,0,1
6,0,1,171000.000,1560726.0,41301.0,0.774761,0.724000,0.492060,-13778,-3130,0,1,0,1
8,0,0,112500.000,1019610.0,33826.5,0.587334,0.205747,0.751724,-20099,365243,1,0,0,1
10,0,1,112500.000,652500.0,21177.0,0.319760,0.651862,0.363945,-10197,-679,1,0,0,1
11,0,0,38419.155,148365.0,10678.5,0.722044,0.555183,0.652897,-20417,365243,1,0,0,1


In [53]:
X_train, X_test, y_train, y_test = train_test_split(numerical_df.drop('TARGET', 1), 
                                    numerical_df['TARGET'], test_size = .2, random_state=42,shuffle=True)

In [54]:
# SMOTE: Overclassing Minority Classification

sum_1 = sum(y_train==1)
sum_0 = sum(y_train==0)

print('Total Data', len(y_train))
print('Imbalance: [-]', sum_0, '[+]', sum_1)

print("\nPerforming SMOTE on training data set\n")
sm = SMOTE(random_state=2)
X_train_res, y_train_res = sm.fit_sample(X_train, y_train.ravel())

sum_1 = sum(y_train_res==1)
sum_0 = sum(y_train_res==0)

print('Total Data', len(y_train_res))
print('Imbalance: [-]', sum_0, '[+]', sum_1)

Total Data 87665
Imbalance: [-] 81269 [+] 6396

Performing SMOTE on training data set

Total Data 162538
Imbalance: [-] 81269 [+] 81269


In [55]:
#amount of rows in input data
X_train_res.shape[0]

162538

In [56]:
#amount of columns in input data
len(X_train_res.columns)

13

In [57]:
model = DecisionTreeClassifier()
model.fit(X_train_res, y_train_res)
y_pred=model.predict(X_test)

In [58]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.8044440388739335


In [59]:
print(confusion_matrix(y_test,y_pred))

[[17224  3091]
 [ 1195   407]]


In [60]:
print("Decision Tree TRAIN score with ",format(model.score(X_train_res, y_train_res)))
print("Decision Tree TEST score with ",format(model.score(X_test, y_test)))

Decision Tree TRAIN score with  1.0
Decision Tree TEST score with  0.8044440388739335


In [ ]:
auc = roc_auc_score(y_test, y_pred)
auc

In [ ]:
#cross_validate
cv_results = cross_validate(model, X_train_res, y_train_res, cv=10)

In [32]:
#score with cross validation
np.mean(cv_results['test_score'])

0.7861424157784472

In [33]:
param_grid = {"criterion": ['entropy', 'gini'],
"min_samples_split": [5,10,15],
"max_depth": [2,3,5],
"min_samples_leaf": [5,10,15],
"max_leaf_nodes": [5,10,15],
}
CV_rfc = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)
CV_rfc.fit(X_train_res, y_train_res)
CV_rfc.best_params_

{'criterion': 'gini',
 'max_depth': 5,
 'max_leaf_nodes': 15,
 'min_samples_leaf': 5,
 'min_samples_split': 5}

In [34]:
model = DecisionTreeClassifier(criterion='entropy', max_depth=2, 
                               max_leaf_nodes=5, min_samples_leaf=5, min_samples_split=5)
model.fit(X_train_res, y_train_res)
y_pred=model.predict(X_test)

In [35]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.6660126842177305


In [36]:
print("Decision Tree TRAIN score with ",format(model.score(X_train_res, y_train_res)))
print("Decision Tree TEST score with ",format(model.score(X_test, y_test)))

Decision Tree TRAIN score with  0.6879314375715218
Decision Tree TEST score with  0.6660126842177305


In [37]:
print(confusion_matrix(y_test,y_pred))

[[13688  6627]
 [  693   909]]


In [38]:
auc = roc_auc_score(y_test, y_pred)
auc

0.620601785916755